In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import re
import time
import json

In [35]:
# Обязательная часть
# Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
# Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое 
# слово. Эти слова определяем в начале кода в переменной, например:
# KEYWORDS = ['python', 'парсинг']
# Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно 
# с текущей страницы).
# В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

KEYWORDS = ['python', 'парсинг', '9to5Mac', 'CrowdSec', 'Kotlin', 'проблёскивающей']

In [3]:
req = requests.get('http://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
content_list = soup.find_all('li', {'id': re.compile('post_\d')}) # все превьюхи собираются
keywords_content_list = []

for content in content_list: # отбираются превьюхи в которых фигурируют ключевые слова
    if any(keyword in str(content) for keyword in KEYWORDS):
        keywords_content_list.append(content)

len(keywords_content_list)

2

In [5]:
df = pd.DataFrame()
now = datetime.now()

for content in keywords_content_list:
    date = content.find(class_=['post__time', 'preview-data__time-published']).text
    if 'вчера' in date: # дата меняется на фактическую без "вчера", "сегодня"
        date = '{:%Y-%m-}{} '.format(now, now.day - 1) + date[8:]
    elif 'сегодня' in date:
        date = '{:%Y-%m-}{} '.format(now, now.day) + date[10:]
    header = content.find(class_=['post__title_link', 'preview-data__title-link']).text
    link = content.find(class_="post__title_link").get('href')
    
    row = {'date': date, 'header': header, 'link': link}
    df = pd.concat([df, pd.DataFrame([row])])

df.reset_index(drop=True, inplace=True)
df

,date,header,link
0,2021-01-16 22:35,"Тестирование Kotlin/JS: фреймворки, корутины и...",https://habr.com/ru/post/537670/
1,2021-01-16 17:22,Как определять собственные классы исключений в...,https://habr.com/ru/company/piter/blog/537642/


In [38]:
# Дополнительная часть (необязательная)
# Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, 
# но и весь текст статьи целиком.
# Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
# Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

content_list = soup.find_all('li', {'id': re.compile('post_\d')}) # все превьюхи собираются

df = pd.DataFrame()
now = datetime.now()

for content in content_list:
    date = content.find(class_=['post__time', 'preview-data__time-published']).text
    link = content.find(class_="post__title_link").get('href')
    tmpreq = requests.get(link)
    tmpsoup = BeautifulSoup(tmpreq.text, 'html.parser')
    post_text = tmpsoup.find(class_='post__wrapper').text
    if any(keyword in str(content) for keyword in KEYWORDS):
        if 'вчера' in date: # дата меняется на фактическую без "вчера", "сегодня"
            date = '{:%Y-%m-}{} '.format(now, now.day - 1) + date[8:]
        elif 'сегодня' in date:
            date = '{:%Y-%m-}{} '.format(now, now.day) + date[10:]
        header = content.find(class_=['post__title_link', 'preview-data__title-link']).text
        row = {'date': date, 'header': header, 'link': link, 'text': post_text}
        df = pd.concat([df, pd.DataFrame([row])])

df.reset_index(drop=True, inplace=True)
df

,date,header,link,text
0,2021-01-17 10:21,Черные дыры и голограммы,https://habr.com/ru/post/537500/,\n\n\n\nArkadiyXIII\n\nсегодня в 10:21\n\n\nЧе...
1,2021-01-16 22:35,"Тестирование Kotlin/JS: фреймворки, корутины и...",https://habr.com/ru/post/537670/,\n\n\n\nkamish721\n\nвчера в 22:35\n\n\nТестир...
2,2021-01-16 17:22,Как определять собственные классы исключений в...,https://habr.com/ru/company/piter/blog/537642/,\n\n\n\nph_piter\n\nвчера в 17:22\n\n\nКак опр...


In [18]:
# Задание 2.
# Обязательная часть
# Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса 
# Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
# EMAIL = [xxx@x.ru, yyy@y.com]

# В итоге должен формироваться датафрейм со столбцами: 
# <почта> - <дата утечки> - <источник утечки> - <описание утечки>

# Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [42]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'
    }

params = {'emailAddresses':['iif984@gmail.com', 'dsada@dasdqwe.eeer', 'elkichee@gmail.com']}

req = requests.post(URL, data=json.dumps(params), headers=headers)

In [41]:
soup = BeautifulSoup(req.text, 'html.parser')
response = json.loads(str(soup))

df = pd.DataFrame()

for key, value in response['summary'].items():
    if len(value['breaches']) < 1:
        continue
    else:
        for breach in value['breaches']:
            date = response['breaches'][str(breach)]['publishDate']
            source = response['breaches'][str(breach)]['site']
            description = response['breaches'][str(breach)]['description']
            row = {'e-mail': key, 'date': date, 'source': source, 'description': description}
            df = pd.concat([df, pd.DataFrame([row])])

df.reset_index(drop=True, inplace=True)
df

,e-mail,date,source,description
0,iif984@gmail.com,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
1,elkichee@gmail.com,2016-10-24T00:00:00Z,000webhost.com,An anonymous criminal was able to exploit an o...
2,elkichee@gmail.com,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
3,elkichee@gmail.com,2016-10-23T00:00:00Z,hostinger.com,"In May 2015, Hostinger.com's database was brea..."
4,elkichee@gmail.com,2019-05-23T00:00:00Z,livejournal.com,"In 2017, social network LiveJournal's database..."
